# Robust homeostasis of cellular cholesterol is a consequence of endogenous antithetic integral control
[Scheepers and Araujo (2023)](10.3389/fcell.2023.1244297)

In [1]:
# using ReactionNetworkDecomposition
using Catalyst
using LinearAlgebra
import Symbolics
import Oscar

[ Info: Assuming ((1//128)*(√((5120//1)*(a^4)) - (80//1)*(a^2))) != 0
[ Info: Assuming ((1//128)*(√((5120//1)*(a^4)) - (80//1)*(a^2))) != 0


In [2]:
rn = include("../data/cholesterol_rn.jl")

Model Cholesterol_RN:
Unknowns (14): see unknowns(Cholesterol_RN)
  S_p(t)
  C(t)
  P(t)
  R(t)
  ⋮
Parameters (23): see parameters(Cholesterol_RN)
  k_1
  k_2
  k_3
  k_4
  ⋮

In [3]:
I = polynomial_ideal(rn)

Ideal generated by
  -k_13*S_p + k_1*S_ci
  -η*C*S_ci + θ*C_e
  p_3*S_p - k_16*P
  -k_14*P*R - k_11*R + p_1*S_r
  -k_2*S_r + k_1*S_ci
  -k_10*S_h + k_1*S_ci
  -k_9*H*H_R + α
  p_2*S_h - k_15*H_R*C_e
  -k_5*C_p + k_4*C_f + k_6*C_e
  -k_8*E + k_7*C_e
  -η*C*S_ci + μ
  k_3*R*C_L - k_4*C_f
  k_9*H*H_R + k_5*C_p + k_8*E + (-k_6 - k_7 - k_12)*C_e
  0

### Groebner Basis

In [4]:
@unpack C_f, C_L = rn
GI = elimination_ideal(I, [C_f, C_L])

Gröbner basis with elements
  1: k_4*θ*C_f - k_12*μ + θ*α
  2: θ*C_e - μ
  3: k_1*k_3*k_13*k_16*p_1*θ*S_ci*C_L + (-k_1*k_2*k_12*k_14*p_3*μ + k_1*k_2*k_14*p_3*θ*α)*S_ci - k_2*k_4*k_11*k_13*k_16*θ*C_f
  4: k_8*E - k_7*C_e
  5: k_5*C_p - k_6*C_e - k_4*C_f
  6: k_10*k_15*μ*H_R - k_1*p_2*θ*S_ci
  7: (k_2*k_9*k_11*k_12*k_13*k_16*p_2*μ*θ - k_2*k_9*k_11*k_13*k_16*p_2*θ^2*α)*H + k_3*k_5*k_10*k_13*k_15*k_16*p_1*μ*θ*C_p*C_L + (-k_2*k_5*k_10*k_12*k_14*k_15*p_3*μ^2 + k_2*k_5*k_10*k_14*k_15*p_3*μ*θ*α)*C_p + k_3*k_8*k_10*k_13*k_15*k_16*p_1*μ*θ*E*C_L + (-k_2*k_8*k_10*k_12*k_14*k_15*p_3*μ^2 + k_2*k_8*k_10*k_14*k_15*p_3*μ*θ*α)*E + (-k_3*k_6*k_10*k_13*k_15*k_16*p_1*μ*θ - k_3*k_7*k_10*k_13*k_15*k_16*p_1*μ*θ - k_3*k_10*k_12*k_13*k_15*k_16*p_1*μ*θ)*C_e*C_L + (k_2*k_6*k_10*k_12*k_14*k_15*p_3*μ^2 - k_2*k_6*k_10*k_14*k_15*p_3*μ*θ*α + k_2*k_7*k_10*k_12*k_14*k_15*p_3*μ^2 - k_2*k_7*k_10*k_14*k_15*p_3*μ*θ*α + k_2*k_10*k_12^2*k_14*k_15*p_3*μ^2 - k_2*k_10*k_12*k_14*k_15*p_3*μ*θ*α)*C_e
  8: k_10*S_h - k_1*S_ci
  9: k

In [15]:
params = parameters(rn)
sym_coeffs = lift_symbolics(GI[1], I, params)

14-element Vector{Num}:
     0
  k_12
     0
     0
     0
     0
     θ
     0
     θ
     θ
 -k_12
     0
     θ
     0

In [16]:
odesys = convert(ODESystem, rn)
eqn_lhs = [eqn.lhs for eqn in equations(odesys)]
eqn_rhs = [eqn.rhs for eqn in equations(odesys)]

dot(eqn_lhs, sym_coeffs) |> Symbolics.expand |> display
rho = dot(eqn_rhs, sym_coeffs) |> Symbolics.expand

k_12*Differential(t)(C(t)) - k_12*Differential(t)(S_ci(t)) + Differential(t)(C_p(t))*θ + Differential(t)(E(t))*θ + Differential(t)(C_e(t))*θ + Differential(t)(H(t))*θ

-k_12*μ + α*θ + k_4*C_f(t)*θ

In [18]:
setpoint = Symbolics.symbolic_solve(rho, C_f)[]

(-k_12*μ + α*θ) / (-k_4*θ)

### Robust Perfect Adaptation (RPA)

In [20]:
@unpack C_e, C_p, C_f, E, C_L = rn

for rpa_var in [C_e, C_p, C_f, E]
    GI = elimination_ideal(I, [rpa_var, C_L])
    A = lift_symbolics(GI[1], I, params)
    rho = dot(eqn_rhs, A) |> Symbolics.expand
    setpoint = Symbolics.symbolic_solve(rho, rpa_var)[]
    println("RPA var: $rpa_var")
    println("ρ: $rho")
    println("------------")
    display(rpa_var ~ setpoint)
end

RPA var: C_e(t)
ρ: -μ + C_e(t)*θ
------------
RPA var: C_p(t)
ρ: -k_12*μ - k_6*μ + α*θ + k_5*C_p(t)*θ
------------
RPA var: C_f(t)
ρ: -k_12*μ + α*θ + k_4*C_f(t)*θ
------------
RPA var: E(t)
ρ: -k_7*μ + k_8*E(t)*θ
------------


C_e(t) ~ μ / θ

C_p(t) ~ (-k_12*μ - k_6*μ + α*θ) / (-k_5*θ)

C_f(t) ~ (-k_12*μ + α*θ) / (-k_4*θ)

E(t) ~ (k_7*μ) / (k_8*θ)